<a href="https://colab.research.google.com/github/3dsf/BlendGAN/blob/main/BlendGAN_3dsf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BlendGAN 
Runs examples from 
https://github.com/onion-liu/BlendGAN

*colab notebook adaptation by u/3dsf*

## Use GPU Runtime

In [17]:
import torch
if torch.cuda.is_available() :
  print('CUDA device: ',torch.cuda.get_device_name(0))
else :
  print("Please enable GPU runtime")
  raise SystemExit("EXECUTION stopped because NOt using GPU")

CUDA device:  Tesla K80


## Install & Setup

In [1]:
!git clone https://github.com/onion-liu/BlendGAN.git
!pip install gdown ninja

import os
from google.colab import files
pathBG = '/content/BlendGAN/'

Cloning into 'BlendGAN'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 102 (delta 13), reused 97 (delta 8), pack-reused 0
Receiving objects: 100% (102/102), 51.13 MiB | 28.55 MiB/s, done.
Resolving deltas: 100% (13/13), done.
     |████████████████████████████████| 108 kB 5.0 MB/s 


In [2]:
cpBlendGAN = os.path.join(pathBG, 'pretrained_models/blendgan.pt')
cpPSP      = os.path.join(pathBG, 'pretrained_models/psp_encoder.pt')
cpStyle    = os.path.join(pathBG, 'pretrained_models/style_encoder.pt')

if os.path.exists(cpBlendGAN) == False :
  !gdown --id 1eF04jKMLAb9DvzI72m8Akn5ykWf3EafE -O $cpBlendGAN
if os.path.exists(cpPSP) == False : 
  !gdown --id 14nevG94hNkkwaoK5eJLF1iv78cv5O8fN -O $cpPSP
if os.path.exists(cpStyle) == False :
  !gdown --id 1EaM0ZYsAMdPkbRz0smLNIlJ1rxVAhbEz -O $cpStyle

Downloading...
From: https://drive.google.com/uc?id=1eF04jKMLAb9DvzI72m8Akn5ykWf3EafE
To: /content/BlendGAN/pretrained_models/blendgan.pt
100% 3.28G/3.28G [00:45<00:00, 71.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=14nevG94hNkkwaoK5eJLF1iv78cv5O8fN
To: /content/BlendGAN/pretrained_models/psp_encoder.pt
100% 1.07G/1.07G [00:22<00:00, 47.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EaM0ZYsAMdPkbRz0smLNIlJ1rxVAhbEz
To: /content/BlendGAN/pretrained_models/style_encoder.pt
100% 1.33G/1.33G [00:26<00:00, 50.4MB/s]


# Generate image pairs with random face codes
* First run will download vgg19 and will take longer



## latent-guided generation
* *uses ./test_imgs/face_imgs and ./test_imgs/style_imgs (coded into generate_image_pairs.py)*

In [8]:
N_PICS = 8

###

os.chdir(pathBG)
pathOutputLatentGuided = os.path.join(pathBG, 'results/generated_pairs/latent_guided/')
!python generate_image_pairs.py --size 1024 --pics $N_PICS --ckpt $cpBlendGAN --outdir $pathOutputLatentGuided

os.chdir(pathOutputLatentGuided)
tarOutLG = os.path.join(pathBG, 'latentGuided.tar.gz')
!tar -czvf $tarOutLG *
files.download(tarOutLG)



tcmalloc: large alloc 1249902592 bytes == 0x55d4b3412000 @  0x7f3612403b6b 0x7f3612423379 0x7f34fad0fcde 0x7f34fad11452 0x7f354d783749 0x7f35f65acd59 0x55d4a7aa9045 0x55d4a7a69c52 0x55d4a7adcc25 0x55d4a7ad7ced 0x55d4a7a6a7f3 0x55d4a7a6a2f9 0x55d4a7bb135d 0x55d4a7b20a0b 0x55d4a7a693a1 0x55d4a7b5ae1d 0x55d4a7adce99 0x55d4a7ad7ced 0x55d4a79a9e2b 0x55d4a7ad9fe4 0x55d4a7ad79ee 0x55d4a7a6abda 0x55d4a7adcd00 0x55d4a7ad79ee 0x55d4a7ad76f3 0x55d4a7ba14c2 0x55d4a7ba183d 0x55d4a7ba16e6 0x55d4a7b79163 0x55d4a7b78e0c 0x7f361120bbf7
tcmalloc: large alloc 1249902592 bytes == 0x55d51ef2a000 @  0x7f3612403b6b 0x7f3612423379 0x7f34fad0fcde 0x7f34fad11452 0x7f354d783749 0x7f35f65acd59 0x55d4a7aa9045 0x55d4a7a69c52 0x55d4a7adcc25 0x55d4a7ad7ced 0x55d4a7a6a7f3 0x55d4a7a6a2f9 0x55d4a7bb135d 0x55d4a7b20a0b 0x55d4a7a693a1 0x55d4a7b5ae1d 0x55d4a7adce99 0x55d4a7ad7ced 0x55d4a79a9e2b 0x55d4a7ad9fe4 0x55d4a7ad79ee 0x55d4a7a6abda 0x55d4a7adcd00 0x55d4a7ad79ee 0x55d4a7ad76f3 0x55d4a7ba14c2 0x55d4a7ba183d 0x55d4a7ba

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## reference-guided generation

* *uses ./test_imgs/face_imgs  (coded into generate_image_pairs.py)*

In [4]:
N_PICS = 8
refStyleImage = os.path.join(pathBG, 'test_imgs/style_imgs/100036.png')

###

os.chdir(pathBG)
pathOutputReferenceGuided = os.path.join(pathBG, '/results/generated_pairs/reference_guided/')
!python generate_image_pairs.py --size 1024 --pics $N_PICS --ckpt $cpBlendGAN --style_img $refStyleImage --outdir $pathOutputReferenceGuided

os.chdir(pathOutputReferenceGuided)
tarOutRG = os.path.join(pathBG, 'referenceGuided.tar.gz')
!tar -czvf $tarOutRG *
files.download(tarOutRG)



tcmalloc: large alloc 1249902592 bytes == 0x55eee1df2000 @  0x7fea7c3eab6b 0x7fea7c40a379 0x7fe964cf6cde 0x7fe964cf8452 0x7fe9b776a749 0x7fea60593d59 0x55eed80e2045 0x55eed80a2c52 0x55eed8115c25 0x55eed8110ced 0x55eed80a37f3 0x55eed80a32f9 0x55eed81ea35d 0x55eed8159a0b 0x55eed80a23a1 0x55eed8193e1d 0x55eed8115e99 0x55eed8110ced 0x55eed7fe2e2b 0x55eed8112fe4 0x55eed81109ee 0x55eed80a3bda 0x55eed8115d00 0x55eed81109ee 0x55eed81106f3 0x55eed81da4c2 0x55eed81da83d 0x55eed81da6e6 0x55eed81b2163 0x55eed81b1e0c 0x7fea7b1f2bf7
tcmalloc: large alloc 1249902592 bytes == 0x55ef4d90a000 @  0x7fea7c3eab6b 0x7fea7c40a379 0x7fe964cf6cde 0x7fe964cf8452 0x7fe9b776a749 0x7fea60593d59 0x55eed80e2045 0x55eed80a2c52 0x55eed8115c25 0x55eed8110ced 0x55eed80a37f3 0x55eed80a32f9 0x55eed81ea35d 0x55eed8159a0b 0x55eed80a23a1 0x55eed8193e1d 0x55eed8115e99 0x55eed8110ced 0x55eed7fe2e2b 0x55eed8112fe4 0x55eed81109ee 0x55eed80a3bda 0x55eed8115d00 0x55eed81109ee 0x55eed81106f3 0x55eed81da4c2 0x55eed81da83d 0x55eed81d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Style tranfer with given face images

In [10]:
pathStyleImgs = '/content/BlendGAN/test_imgs/style_imgs/'
pathInputImgs = '/content/BlendGAN/test_imgs/face_imgs/'
pathOutputStyleTransfer = '/content/BlendGAN/results/style_transfer/'

os.chdir(pathBG)
!python style_transfer_folder.py --size 1024 --ckpt $cpBlendGAN --psp_encoder_ckpt $cpPSP --style_img_path $pathStyleImgs --input_img_path $pathInputImgs --outdir $pathOutputStyleTransfer

os.chdir(pathOutputStyleTransfer)
tarOutST = os.path.join(pathBG, 'styleTransfer.tar.gz')
!tar -czvf $tarOutST *
files.download(tarOutST)

tcmalloc: large alloc 1249902592 bytes == 0x55f2c4f82000 @  0x7ff2c5201b6b 0x7ff2c5221379 0x7ff1adb0dcde 0x7ff1adb0f452 0x7ff200581749 0x7ff2a93aad59 0x55f2bac4f045 0x55f2bac0fc52 0x55f2bac82c25 0x55f2bac7dced 0x55f2bac107f3 0x55f2bac102f9 0x55f2bad5735d 0x55f2bacc6a0b 0x55f2bac0f3a1 0x55f2bad00e1d 0x55f2bac82e99 0x55f2bac7dced 0x55f2bab4fe2b 0x55f2bac7ffe4 0x55f2bac7d9ee 0x55f2bac10bda 0x55f2bac82d00 0x55f2bac7d9ee 0x55f2bac7d6f3 0x55f2bad474c2 0x55f2bad4783d 0x55f2bad476e6 0x55f2bad1f163 0x55f2bad1ee0c 0x7ff2c4009bf7
tcmalloc: large alloc 1249902592 bytes == 0x55f330a9a000 @  0x7ff2c5201b6b 0x7ff2c5221379 0x7ff1adb0dcde 0x7ff1adb0f452 0x7ff200581749 0x7ff2a93aad59 0x55f2bac4f045 0x55f2bac0fc52 0x55f2bac82c25 0x55f2bac7dced 0x55f2bac107f3 0x55f2bac102f9 0x55f2bad5735d 0x55f2bacc6a0b 0x55f2bac0f3a1 0x55f2bad00e1d 0x55f2bac82e99 0x55f2bac7dced 0x55f2bab4fe2b 0x55f2bac7ffe4 0x55f2bac7d9ee 0x55f2bac10bda 0x55f2bac82d00 0x55f2bac7d9ee 0x55f2bac7d6f3 0x55f2bad474c2 0x55f2bad4783d 0x55f2bad4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ~Generate interpolation videos~
#### Crashes when writing video in colab and during local testing (RAM overflow)

*uncomment next cell to try*

In [ ]:
#os.chdir(pathBG)
#!python gen_video.py --size 1024 --ckpt $cpBlendGAN --psp_encoder_ckpt $cpPSP --style_img_path ./test_imgs/style_imgs/ --input_img_path ./test_imgs/face_imgs/ --outdir results/inter_videos/